In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [10]:
#load data
df = pd.read_csv('u.data', sep='\t', names=['user','item','rate', 'time'])
#<<<Problem of this model: "overflow encountered" -> here we just test with small amount of data
df = df[:10000]
#>>>
msk = np.random.rand(len(df)) < 0.7
df_train = df[msk]

user_indecies = [x for x in df_train.user.values]
item_indecies = [x for x in df_train.item.values]
rates = df_train.rate.values

In [11]:
print(len(df_train))
print(df.groupby('user')['item'].nunique().count())
print(df.groupby('item')['user'].nunique().count())
print(df_train.groupby('user')['item'].nunique().count())
print(df_train.groupby('item')['user'].nunique().count())
print(max(df_train['user']))
print(max(df_train['item']))

7011
385
1254
382
1170
388
1529


In [12]:
R_df = df_train.pivot(index = 'user', columns ='item', values = 'rate').convert_objects(convert_numeric=True).fillna(0)
R_df.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


item,1,2,3,4,5,6,7,8,9,10,...,1480,1489,1495,1497,1500,1503,1508,1518,1520,1529
user,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R_demeaned

array([[-0.19487179, -0.19487179, -0.19487179, ..., -0.19487179,
        -0.19487179, -0.19487179],
       [-0.02905983, -0.02905983, -0.02905983, ..., -0.02905983,
        -0.02905983, -0.02905983],
       [-0.02136752, -0.02136752, -0.02136752, ..., -0.02136752,
        -0.02136752, -0.02136752],
       ...,
       [-0.0025641 , -0.0025641 , -0.0025641 , ..., -0.0025641 ,
        -0.0025641 , -0.0025641 ],
       [-0.01880342, -0.01880342, -0.01880342, ..., -0.01880342,
        -0.01880342, -0.01880342],
       [-0.0042735 , -0.0042735 , -0.0042735 , ..., -0.0042735 ,
        -0.0042735 , -0.0042735 ]])

In [6]:
class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [15]:
mf = MF(R_demeaned, K=2, alpha=0.1, beta=0.01, iterations=5)
mf.train()
preds_df = pd.DataFrame(mf.full_matrix(), columns = R_df.columns)

In [16]:
u, p, r = df[['user', 'item', 'rate']].values[0]
rhat = preds_df[1444][0]
print ( str(r) + '  ' + str(rhat))

3  3.093509973165808


In [17]:
#accuracy
df_test = df[~msk]

compare_rate = []
for i in range(len(df_test)):
    u, p, r = df_test[['user', 'item', 'rate']].values[i]
    if u in preds_df.index and p in preds_df.columns:
        pred = preds_df[p][u]
        compare_rate.append(dict(user= u, item= p, rate= r, pred= pred))

sum_abs = 0
for x in compare_rate:
    sum_abs += abs(x['rate'] - x['pred'])
sum_abs/len(compare_rate)

1.1551258997883933